# ლექცია #12

## კოდების გაშვებისთვის საჭირო ბიბლიოთეკების იმპორტები

ამ ნოუთბუქის გაშვებამდე არ დაგავიწყდეთ საჭირო დამოკიდებულებების (dependencies) დაინსტალირება, რომლებიც მოცემულია `requirements.txt` ფაილში. მარტივად, ტერმინალიდან გაუშვით:

```bash
pip install -r requirements.txt
```

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    train_test_split,
)

## მონაცემების ჩატვირთვა

In [ ]:
iris_data = load_iris()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    iris_data.data,
    iris_data.target,
    test_size=0.2,
    random_state=1,
    stratify=iris_data.target,
)

In [ ]:
print("სატრენინგო მონაცემების ზომა:", X_train.shape, y_train.shape)
print("სატესტო მონაცემების ზომა:", X_test.shape, y_test.shape)

## ჯვარედინი ვალიდაცია

ჯვარედინი ვალიდაცია ML-ში არის ტექნიკა, რომელიც გამოიყენება მოდელის პერფორმანსის და განზოგადების შესაფასებლად. იგი მოიცავს მონაცემთა ნაკრების მრავალ ქვეჯგუფად დაყოფას, მოდელის დატრენინგებას ზოგიერთ ქვეჯგუფზე (სატრენინგო მონაცემები) და მის ვალიდაციას დანარჩენ ქვეჯგუფზე (ვალიდაციის მონაცემები). ეს პროცესი რამდენჯერმე მეორდება სხვადასხვა ქვეჯგუფებთან, რათა უზრუნველყოს მოდელის სტაბილურობა და შემცირდეს "ზედმეტად მორგება".

სხვადასხვა ტიპის ჯვარედინი ვალიდაციის ტექნიკა არსებობს:

* **k-Fold**: მონაცემები დაყოფილია k თანაბარი ზომის ნაწილად და თითოეული ნაწილი გამოიყენება ვალიდაციის მონაცემებად ერთხელ, ხოლო დარჩენილი k-1 ნაწილები გამოიყენება ტრენინგისთვის.
* **Stratified k-Fold Cross-Validation**: k-fold-ის მსგავსია, თუმცა უზრუნველყოფს, რომ თითოეულ ნაწილში კლასების იგივე პროპორცია იყოს, როგორც თავდაპირველ მონაცემთა ნაკრებში.
* **Repeated k-Fold Cross-Validation**: k-fold ჯვარედინი ვალიდაციის პროცესი მეორდება რამდენჯერმე, თუმცა ყოველ ჯერზე მონაცემების სხვადასხვა შემთხვევითი გაყოფით.
* **Leave-One-Out Cross-Validation (LOOCV)**: თითოეული დაკვირვება/მაგალითი მონაცემთა ნაკრებში გამოიყენება ვალიდაციის მონაცემად ერთხელ და დანარჩენი მაგალითები გამოიყენება სატრენინგო მონაცემებად.
* **Leave-p-Out Cross-Validation (LpOCV)**: LOOCV-ის მსგავსია, თუმცა ერთი დაკვირვების ნაცვლად, p რაოდენობის მაგალითია გამოყენებული ვალიდაციისთვის და მოდელი ტრენინგდება დანარჩენ დაკვირვებებზე.
* **Time Series Cross-Validation**: გამოიყენება დროით მწკრივებზე, სადაც მონაცემების თანმიმდევრობა მნიშვნელოვანია. მონაცემები იყოფა ისე, რომ ტრენინგი ხდება წარსულ მონაცემებზე, ხოლო ვალიდაცია - მომავლის/ახლანდელ მონაცემებზე.

### Stratified k-Fold Cross-Validation

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

train_accuracies = []
valid_accuracies = []

for fold_n, (train_idx, valid_idx) in enumerate(skf.split(X_train, y_train), start=1):
    print(f"Fold #{fold_n}\n")

    X_train_new, X_valid = X_train[train_idx], X_train[valid_idx]
    y_train_new, y_valid = y_train[train_idx], y_train[valid_idx]

    log_reg = LogisticRegression(random_state=1)
    log_reg.fit(X_train_new, y_train_new)

    train_preds = log_reg.predict(X_train_new)
    train_score = accuracy_score(y_train_new, train_preds)
    print("სატრენინგო მონაცემები:")
    print(f"\tდაკვირვებების რაოდენობა: {len(y_train_new)}")
    print(f"\tSetosa-ს რაოდენობა: {(y_train_new == 0).sum()}")
    print(f"\tVersicolor-ის რაოდენობა: {(y_train_new == 1).sum()}")
    print(f"\tVirginica-ს რაოდენობა: {(y_train_new == 2).sum()}")
    print(f"\tაკურატულობა: {train_score * 100:.2f}%")
    train_accuracies.append(train_score)

    print()

    valid_preds = log_reg.predict(X_valid)
    valid_score = accuracy_score(y_valid, valid_preds)
    print("ვალიდაციის მონაცემები:")
    print(f"\tდაკვირვებების რაოდენობა: {len(y_valid)}")
    print(f"\tSetosa-ს რაოდენობა: {(y_valid == 0).sum()}")
    print(f"\tVersicolor-ის რაოდენობა: {(y_valid == 1).sum()}")
    print(f"\tVirginica-ს რაოდენობა: {(y_valid == 2).sum()}")
    print(f"\tაკურატულობა: {valid_score * 100:.2f}%")
    valid_accuracies.append(valid_score)

    print()

mean_train_score = sum(train_accuracies) / len(train_accuracies)
mean_valid_score = sum(valid_accuracies) / len(valid_accuracies)

print(f"საშუალო აკურატულობა სატრენინგო მონაცემებზე: {mean_train_score * 100:.2f}%")
print(f"საშუალო აკურატულობა ვალიდაციის მონაცემებზე: {mean_valid_score * 100:.2f}%")

## ჰიპერპარამეტრების ოპტიმიზაცია (Hyperparameter Optimization - HPO)

**ჰიპერპარამეტრები** არის პარამეტრები, რომლებიც უნდა განისაზღვროს ტრენინგის პროცესის დაწყებამდე. ისინი არ არის ნასწავლი მონაცემებიდან, არამედ დეველოპერის/მეცნიერის მიერ არის განსაზღვრული. ჰიპერპარამეტრების მაგალითებია `n_estimators`, `max_depth` და `max_features` `RandomForestClassifier`-ში.

ჰიპერპარამეტრების ოპტიმიზაცია არის ML მოდელისთვის ჰიპერპარამეტრების საუკეთესო ნაკრების პოვნის პროცესი. მისი მიზანია მოდელის მუშაობის გაუმჯობესება ყველაზე შესაფერისი/ოპტიმალური ჰიპერპარამეტრების შერჩევით.

In [ ]:
X_train_new, X_valid, y_train_new, y_valid = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=1,
    stratify=y_train,
)

In [ ]:
print("სატრენინგო მონაცემების ზომა:", X_train_new.shape, y_train_new.shape)
print("ვალიდაციის მონაცემების ზომა:", X_valid.shape, y_valid.shape)

In [ ]:
best_score = 0
train_score = 0
best_params = None

for n_estimators in range(3, 6):
    for max_features in ["log2", "sqrt", None]:
        for max_depth in range(3, 6):
            rf = RandomForestClassifier(
                random_state=1,
                n_estimators=n_estimators,
                max_features=max_features,
                max_depth=max_depth,
            )
            rf.fit(X_train_new, y_train_new)

            valid_score = accuracy_score(y_valid, rf.predict(X_valid))
            if valid_score > best_score:
                train_score = accuracy_score(y_train_new, rf.predict(X_train_new))
                best_score = valid_score
                best_params = {
                    "n_estimators": n_estimators,
                    "max_features": max_features,
                    "max_depth": max_depth,
                }
print(f"საუკეთესო აკურატულობა ვალიდაციის მონაცემებზე: {best_score * 100:.2f}%")
print(f"აკურატულობა სატრენინგო მონაცემებზე: {train_score * 100:.2f}%")
print(f"ოპტიმალური პარამეტრები: {best_params}")

### GridSearchCV

"Grid Search with Cross-Validation" შეგვიძლია გამოვიყენოთ მოდელის ჰიპერპარამეტრების ოპტიმიზაციისთვის. როგორც სახელი მიანიშნებს, ის იყენებს ჯვარედინ ვალიდაციას. "Grid" მიუთითებს ამომწურავ ძიებაზე ჰიპერპარამეტრული სივრცის ხელით მითითებულ მნიშვნელობებზე. მაგალითად, თუ გვაქვს ორი ჰიპერპარამეტრი სამი შესაძლო მნიშვნელობით, `GridSearchCV` შეაფასებს ამ ჰიპერპარამეტრების $3 \times 3 = 9$ კომბინაციას. 

In [ ]:
params = {
    "n_estimators": range(3, 6),
    "max_features": ["log2", "sqrt", None],
    "max_depth": range(3, 6),
}

rf = RandomForestClassifier(random_state=1)
grid = GridSearchCV(
    estimator=rf,
    param_grid=params,
    scoring="accuracy",
    cv=3,
    verbose=1,
    return_train_score=True,
    refit=True,
)
grid.fit(X_train, y_train)

In [ ]:
print(f"საუკეთესო საშუალო აკურატულობა: {grid.best_score_ * 100:.2f}%")
print(f"ოპტიმალური ჰიპერპარამეტრები: {grid.best_params_}")

In [ ]:
print(
    "საუკეთესო მოდელის საშუალო აკურატულობა სატრენინგო მონაცემებზე: "
    f"{grid.cv_results_['mean_train_score'][grid.best_index_] * 100:.2f}%"
)
print(
    "საუკეთესო მოდელის საშუალო აკურატულობა ვალიდაციის მონაცემებზე: "
    f"{grid.cv_results_['mean_test_score'][grid.best_index_] * 100:.2f}%"
)

რადგანაც `refit`-ში გადავეცით True, ეს ნიშნავს, რომ საუკეთესო მოდელი ხელახლა დატრენინგდა მთლიანი მონაცემებით ოპტიმალური ჰიპერპარამეტრების გამოყენებით. თუმცა შეგვიძლია თავადაც დავატრენინგოთ საბოლოო მოდელი.

In [ ]:
rf = RandomForestClassifier(random_state=1, **grid.best_params_)
rf.fit(X_train, y_train)

In [ ]:
grid.best_estimator_.predict(X_test) == rf.predict(X_test)

როგორც ვხედავთ, პროგნოზები სატესტო მონაცემებზე იდენტურია, რადგანაც რეალურად ერთი და იმავე პარამეტრებით დატრენინგებული მოდელები გვაქვს.

### RandomizedSearchCV

"Randomized Search with Cross-Validation" `GridSearchCV`-ის მსგავსად შეგვიძლია გამოვიყენოთ მოდელის ჰიპერპარამეტრების ოპტიმიზაციისთვის ჯვარედინი ვალიდაციის გამოყენებით. თუმცა, იმის ნაცვლად, რომ სცადოს ჰიპერპარამეტრების ყველა შესაძლო კომბინაცია (როგორც `GridSearchCV`-ში), ის ამოწმებს ჰიპერპარამეტრების კომბინაციების ფიქსირებულ რაოდენობას მითითებული განაწილებიდან. შესაბამისად, უფრო ეფექტურია, როდესაც უზარმაზარი ჰიპერპარამეტრების სივრციდან გვსურს ოპტიმალური პარამეტრების ამორჩევა.

In [ ]:
params = {
    "n_estimators": range(3, 6),
    "max_features": ["log2", "sqrt", None],
    "max_depth": range(3, 6),
}

rf = RandomForestClassifier(random_state=1)
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=params,
    scoring="accuracy",
    cv=3,
    n_iter=15,
    verbose=1,
    return_train_score=True,
    random_state=1,
)
random_search.fit(X_train, y_train)

In [ ]:
print(f"საუკეთესო საშუალო აკურატულობა: {random_search.best_score_ * 100:.2f}%")
print(f"ოპტიმალური ჰიპერპარამეტრები: {random_search.best_params_}")

In [ ]:
print(
    "საუკეთესო მოდელის საშუალო აკურატულობა სატრენინგო მონაცემებზე: "
    f"{random_search.cv_results_['mean_train_score'][random_search.best_index_] * 100:.2f}%"
)
print(
    "საუკეთესო მოდელის საშუალო აკურატულობა ვალიდაციის მონაცემებზე: "
    f"{random_search.cv_results_['mean_test_score'][random_search.best_index_] * 100:.2f}%"
)

აქაც `refit`-ში გადაცემული True-ს გამო საუკეთესო მოდელი ხელახლა დატრენინგდა მთლიანი მონაცემებით ოპტიმალური ჰიპერპარამეტრების გამოყენებით:

In [ ]:
rf = RandomForestClassifier(random_state=1, **random_search.best_params_)
rf.fit(X_train, y_train)

In [ ]:
random_search.best_estimator_.predict(X_test) == rf.predict(X_test)

### საბოლოო მოდელის განზოგადების ნახვა

როდესაც HPO-თი ვიპოვით საუკეთესო ჰიპერპარამეტრებს, ამ პარამეტრებით უნდა დატრენინგდეს საბოლოო მოდელი და რა თქმა უნდა, უნდა ვნახოთ მისი განზოგადების უნარი უნახავ, გადადებულ სატესტო, მონაცემებზე:

In [ ]:
rf = RandomForestClassifier(
    random_state=1, n_estimators=3, max_features="log2", max_depth=5
)
rf.fit(X_train, y_train)

In [ ]:
train_preds = rf.predict(X_train)
test_preds = rf.predict(X_test)

print(
    "აკურატულობა სატრენინგო მონაცემებზე: "
    f"{accuracy_score(y_train, train_preds) * 100:.2f}%"
)
print(
    "აკურატულობა სატესტო მონაცემებზე: "
    f"{accuracy_score(y_test, test_preds) * 100:.2f}%"
)